In [1]:
import os
import traceback
import logging
import os.path
import networkx as nx
import pandas as pd
import numpy as np
import pylab as pl
from matplotlib import collections  as mc
import glob

In [2]:
# simversion = '_v01'
simversion = '_plasmid_v01'
# simversion = '_plasmid_v02'
# simversion = '_plasmid_v03'
# simversion = '_plasmid_v04'
# 1 True, 0 False
prepare_database_4_multicsar = 0
run_art = 0 
run_spades = 0
run_panta = 0
split_paralogs = 1
run_multicsar = 0
run_ragout = 0
run_pangraph = 1
# pangenome_data = '/data/hoan/amromics/data/ncbi/Kp100random/' # for pangraph, Ragout, multi-CSAR #v3
pangenome_data = '/data/hoan/amromics/data/ncbi/Ecoli50random/'
# pangenome_data = '/data/hoan/amromics/data/ncbi/Kptest/' # for pangraph, Ragout, multi-CSAR
scaffold_out_dir = '/data/hoan/amromics/genome-graph/scaffold_output/'

In [3]:
if prepare_database_4_multicsar==1:
    os.system('rm -r ' + pangenome_data + 'fna/')
    os.system('mkdir ' + pangenome_data + 'fna/')
    os.system('cp -r '+ pangenome_data + '*.fna.gz '+ pangenome_data + 'fna/')
    os.system('~/miniconda3/envs/panta/bin/gzip -d ' + pangenome_data + 'fna/*.fna.gz')

### Simulate reads using ART

In [4]:
# https://github.com/scchess/Art/blob/master/art_illumina_README

In [5]:
sim_dir = '/data/hoan/amromics/simulation/'
art_bin = '/data/hoan/amromics/simulation/art_bin/./art_illumina'

In [6]:
# ref_data = '/data/hoan/amromics/simulation/references/GCF_000240185.1_ASM24018v2_genomic.fasta'
# ref_data = '/data/hoan/amromics/simulation/references/GCF_000240185.1_ASM24018v2_genomic.fna'
ref_data = '/data/hoan/amromics/data/ncbi/Ecolitest/GCF_002860105.1_ASM286010v1_genomic.fna'
# ref_data = '/data/hoan/amromics/data/ncbi/Ecolitest/GCF_002899495.1_ASM289949v1_genomic.fna'
# ref_data = '/data/hoan/amromics/data/ncbi/Ecolitest/GCF_004135915.1_ASM413591v1_genomic.fna'
sim_name = ref_data.split("/")[-2] + '_' + ref_data.split("/")[-1][:-1][:13]
sim_output = '/data/hoan/amromics/simulation/art_output/sim_reads_' + sim_name + simversion[-4:]
# sim_output = '/data/hoan/amromics/simulation/art_output/paired_dat_test'

In [7]:
sim_output

'/data/hoan/amromics/simulation/art_output/sim_reads_Ecolitest_GCF_002860105_v01'

In [8]:
if run_art: 
    if simversion[-3:]=='v01':
        os.system(art_bin+' -ss HS20 -sam -i '+ref_data+' -p -l 100 -f 70 -m 400 -s 60 -o '+sim_output) # _v01
    if simversion[-3:]=='v02':
        os.system(art_bin+' -ss HS20 -sam -i '+ref_data+' -p -l 100 -f 70 -m 400 -s 10 -o '+sim_output) # _v02
    if simversion[-3:]=='v03':
        os.system(art_bin+' -ss MSv3 -sam -i '+ref_data+' -p -l 250 -f 70 -m 400 -s 60 -o '+sim_output) # _v03
    if simversion[-3:]=='v04':
        os.system(art_bin+' -ss MSv3 -sam -i '+ref_data+' -p -l 250 -f 70 -m 400 -s 10 -o '+sim_output) # _v04

### Run SPADES assembly

In [9]:
spades_output = '/data/hoan/amromics/simulation/art_output/' + 'spades_output_' + sim_name + simversion[-4:]
spades_output

'/data/hoan/amromics/simulation/art_output/spades_output_Ecolitest_GCF_002860105_v01'

In [10]:
if run_spades:
    spades_bin ='~/miniconda3/envs/amromics/bin/spades.py'
    os.system(spades_bin+' --isolate -1 '+sim_output+'1.fq -2 '+sim_output+'2.fq -t 50 -o '+spades_output)

### Run panta

In [11]:
simversionRef = '_'+ pangenome_data.split('/')[-2] +'_'+ sim_name + simversion
simversionRef

'_Ecoli50random_Ecolitest_GCF_002860105_plasmid_v01'

In [12]:
# move spades ouput to panta dir
print('Pangenome data: ', pangenome_data)
if run_panta:
    os.system('cp '+spades_output+'/contigs.fasta ' + pangenome_data + 'g1.fna')
    os.system('rm ' + pangenome_data + 'g1.fna.gz')
    os.system('~/miniconda3/envs/panta/bin/gzip -k ' + pangenome_data + 'g1.fna')

Pangenome data:  /data/hoan/amromics/data/ncbi/Ecoli50random/


In [13]:
# Run panta (change output dir)
# https://askubuntu.com/questions/1252439/not-able-to-activate-conda-environment-through-os-system-command-in-python
conda_dir = 'source ~/miniconda3/etc/profile.d/conda.sh && conda activate panta && '
panta_bin =conda_dir+'python /data/hoan/amromics/panta/panta.py'
panta_output = '/data/hoan/amromics/panta/examples/test/pantaOut' + simversionRef
# panta_output = '/data/hoan/amromics/panta/examples/test/output_Kp100' 
if split_paralogs == 0:
    panta_output = panta_output + '_nosplit'
print('PANTA OUTPUT: ', panta_output)
if run_panta:
    if split_paralogs:
        cmd_panta = panta_bin + ' -p init -a ' +pangenome_data+ '*.fna.gz -o '+panta_output +' -as -s -i 85 -c 20 -e 0.01'
    else:
        print("No split ortholog")
        cmd_panta = panta_bin + ' -p init -a ' +pangenome_data+ '*.fna.gz -o '+panta_output +' -as'
    os.system(cmd_panta)

PANTA OUTPUT:  /data/hoan/amromics/panta/examples/test/pantaOut_Ecoli50random_Ecolitest_GCF_002860105_plasmid_v01


### Run pangraph

In [66]:
try:
    %load_ext autoreload
    %autoreload 2
except Exception as e:
    logging.error(traceback.format_exc())
from pangraph import PanGraph

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
# set parameters
data_dir = panta_output 
incomplete_sample_name = 'g1'
assem_dir = spades_output
fasta_gen = 'partial' # 'all', 'partial'

In [68]:
pangraph_output_greedy = scaffold_out_dir + 'pangraph_greedy'+ simversionRef + '.fasta'
pangraph_output_opt = scaffold_out_dir + 'pangraph_opt'+ simversionRef + '.fasta'

In [69]:
pangraph_output_opt

'/data/hoan/amromics/genome-graph/scaffold_output/pangraph_opt_Ecoli50random_Ecolitest_GCF_002860105_plasmid_v01.fasta'

In [70]:
if run_pangraph:
    # https://stackoverflow.com/questions/4990718/how-can-i-write-a-try-except-block-that-catches-all-exceptions
    pangraph = PanGraph(sample_info=None, gene_info=None, gene_position=None)
    try:
        maximum_matching = 'greedy'
        pangraph.run_pangraph_pipeline(data_dir, incomplete_sample_name, assem_dir, fasta_gen, pangraph_output_greedy, maximum_matching)
    except Exception as e:
        logging.error(traceback.format_exc())
        # Logs the error appropriately. 

Should we use target sequence as reference, hence, the strand will be ok
Set minimum on number of nucleotides =  10 NUMBER OF COMPUTED CONTIGS: 261
Clip the matrix 0.2 !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Use modified  assembly graph
Use union graph: overlap + assembly
Reduce the cost by 1/2 if there is no path in the assembly graph
Reduce the cost by 1/2 if there is no path in the assembly graph
Will test this, Ok?
Success! File written


In [71]:
if run_pangraph:
    try:
        maximum_matching = 'opt'
        pangraph.RERUN_pangraph_pipeline(data_dir, incomplete_sample_name, assem_dir, fasta_gen, pangraph_output_opt, maximum_matching)
    except Exception as e:
        logging.error(traceback.format_exc())
        # Logs the error appropriately. 

Reduce the cost by 1/2 if there is no path in the assembly graph
Reduce the cost by 1/2 if there is no path in the assembly graph
Compute maximum matching
Will test this, Ok?
Success! File written


In [72]:
# try:
#     maximum_matching = 'greedy'
#     pangraph_output_greedy = spades_output + '/contigs_concat_'+ maximum_matching+ simversionRef + '.fasta'
#     pangraph.RERUN_pangraph_pipeline(data_dir, incomplete_sample_name, assem_dir, fasta_gen, pangraph_output_opt, maximum_matching)
# except Exception as e:
#     logging.error(traceback.format_exc())
#     # Logs the error appropriately. 

### Run scaffold methods

In [73]:
### Multi-CSAR
conda_dir = 'source ~/miniconda3/etc/profile.d/conda.sh && conda activate py27 && '
csar_bin = conda_dir + '/data/hoan/amromics/assembly/Multi-CSAR/./multi-csar.php'

In [74]:
multicsar_output = scaffold_out_dir + 'multicsar/multicsar' + simversionRef
if run_multicsar:
    os.system('rm ' + pangenome_data + 'fna/g1*')
    os.system(csar_bin +' -t '+pangenome_data+'g1.fna -r '+pangenome_data+'fna/ --nuc -o '+multicsar_output)
    # os.system(csar_bin +' -t '+pangenome_data+'g1.fna -r '+pangenome_data+'*.fna --nuc -o '+multicsar_output)

In [75]:
multicsar_output

'/data/hoan/amromics/genome-graph/scaffold_output/multicsar/multicsar_Ecoli50random_Ecolitest_GCF_002860105_plasmid_v01'

In [76]:
### Ragout
ragout_bin = 'source ~/miniconda3/etc/profile.d/conda.sh && conda activate py27 && ragout '
ragout_output = scaffold_out_dir + 'multicsar/ragout' + simversionRef
if run_ragout:
    # write receipt file
    ref_files_list = glob.glob(pangenome_data+'fna/*')
    n_files = len(ref_files_list)
    receipt_file_dir = '/data/hoan/amromics/assembly/Ragout/kp100'+simversionRef+'.rcp'
    f = open(receipt_file_dir, "w")
    f.write('.references = ')
    for idx in range(n_files-1):
        f.write('r' + str(idx) + ',')
    f.write('r'+str(n_files-1)+'\n')
    f.write(".target = mg1655\n\n")
    for idx in range(n_files):
        f.write('r'+str(idx)+'.fasta =' + ref_files_list[idx] + '\n')
    f.write('mg1655.fasta = ' + spades_output+'/contigs.fasta\n')
    f.close()

In [77]:
if run_ragout:
    os.system(ragout_bin + receipt_file_dir + ' --outdir ' + ragout_output + ' --refine')
    # ragout kp100_v2.rcp --outdir output_Kp100p_v2/ --refine

In [78]:
print(ragout_output)

/data/hoan/amromics/genome-graph/scaffold_output/multicsar/ragout_Ecoli50random_Ecolitest_GCF_002860105_plasmid_v01


### Quast 

In [79]:
quast_output = '/data/hoan/amromics/genome-graph/scaffold_output/quastResults' + simversionRef
quast_output

'/data/hoan/amromics/genome-graph/scaffold_output/quastResults_Ecoli50random_Ecolitest_GCF_002860105_plasmid_v01'

In [80]:
quast_bin = 'python /data/hoan/amromics/spades_quast/quast-5.2.0/quast.py '
spades_output_fasta = spades_output+'/contigs.fasta'
multicsar_output_fasta = multicsar_output +'/multi-csar.nuc.out.fna'
ragout_output_fasta = ragout_output + '/mg1655_scaffolds.fasta'
if 1:
    if os.path.isfile(ragout_output_fasta):
        os.system(quast_bin + pangraph_output_opt+' '+ pangraph_output_greedy+' '+ multicsar_output_fasta+' '+ragout_output_fasta+' '+spades_output_fasta+' '+
        '-l "Pangraph_OPT, Pangraph_Greedy, Multi-CSAR, Ragout, SPADES" '+ '-r '+ref_data+' -o '+ quast_output+' --silent --extensive-mis-size 5000 --local-mis-size 3000')
    else:
        os.system(quast_bin + pangraph_output_opt+' '+ pangraph_output_greedy+' '+ multicsar_output_fasta+' '+spades_output_fasta+' '+
        '-l "Pangraph_OPT, Pangraph_Greedy, Multi-CSAR, SPADES" '+ '-r '+ref_data+' -o '+ quast_output+' --silent --extensive-mis-size 5000 --local-mis-size 3000')

/data/hoan/amromics/spades_quast/quast-5.2.0/quast.py /data/hoan/amromics/genome-graph/scaffold_output/pangraph_opt_Ecoli50random_Ecolitest_GCF_002860105_plasmid_v01.fasta /data/hoan/amromics/genome-graph/scaffold_output/pangraph_greedy_Ecoli50random_Ecolitest_GCF_002860105_plasmid_v01.fasta /data/hoan/amromics/genome-graph/scaffold_output/multicsar/multicsar_Ecoli50random_Ecolitest_GCF_002860105_plasmid_v01/multi-csar.nuc.out.fna /data/hoan/amromics/simulation/art_output/spades_output_Ecolitest_GCF_002860105_v01/contigs.fasta -l Pangraph_OPT, Pangraph_Greedy, Multi-CSAR, SPADES -r /data/hoan/amromics/data/ncbi/Ecolitest/GCF_002860105.1_ASM286010v1_genomic.fna -o /data/hoan/amromics/genome-graph/scaffold_output/quastResults_Ecoli50random_Ecolitest_GCF_002860105_plasmid_v01 --silent --extensive-mis-size 5000 --local-mis-size 3000


System information:
  OS: Linux-3.10.0-1160.15.2.el7.x86_64-x86_64-with-redhat-7.9-Nitrogen (linux_64)
  Python version: 2.7.5
  CPUs number: 72

Started: 20

Unable to parse the pattern
Unable to parse the pattern
Unable to parse the pattern


Done.

2023-01-11 04:33:58
Running Contig analyzer...
Done.

2023-01-11 04:34:01
Running NA-NGA calculation...
Done.

2023-01-11 04:34:02
Running Genome analyzer...
  NOTICE: No file with genomic features were provided. Use the --features option if you want to specify it.

  NOTICE: No file with operons were provided. Use the -O option if you want to specify it.
Done.

NOTICE: Genes are not predicted by default. Use --gene-finding or --glimmer option to enable it.

2023-01-11 04:34:04
Creating large visual summaries...
This may take a while: press Ctrl-C to skip this step..
  1 of 2: Creating PDF with all tables and plots...
  2 of 2: Creating Icarus viewers...
Done

2023-01-11 04:34:06
RESULTS:
  Text versions of total report are saved to /data/hoan/amromics/genome-graph/scaffold_output/quastResults_Ecoli50random_Ecolitest_GCF_002860105_plasmid_v01/report.txt, report.tsv, and report.tex
  Text versions of transposed total report are saved to /data/hoan/amromics/genome-graph/scaffold_o

In [81]:
# # pangraph.H["weight"][0:4]
# iteration = 0
# for u,v,a in pangraph.H.edges(data=True):
#     print(a)
#     iteration += 1
#     if iteration > 10:
#         break;

In [82]:
# nx.write_gml(pangraph.assembly_graph, quast_output+'/assembly_graph.gml')

In [83]:
pangraph.assembly_graph.number_of_edges()

3360

In [84]:
len(list(nx.weakly_connected_components(pangraph.assembly_graph)))

1

In [85]:
# pangraph.weighted_CG.nodes()

In [86]:
len(list(nx.weakly_connected_components(pangraph.H)))

245395

In [87]:
pangraph.H.number_of_nodes(), pangraph.H.number_of_edges()

(250067, 4878)

In [88]:
ref_data

'/data/hoan/amromics/data/ncbi/Ecolitest/GCF_002860105.1_ASM286010v1_genomic.fna'

## Analysis

In [89]:
# minimap2 /data/hoan/amromics/data/ncbi/Ecolitest/GCF_002860105.1_ASM286010v1_genomic.fna /data/hoan/amromics/genome-graph/scaffold_output/pangraph_opt_Ecoli100random_Ecoli_plasmid_v01.fasta > contigs100vsref_mapping_v01a.paf
cmd_minimap = '~/miniconda3/envs/amromics/bin/minimap2 '+ref_data +' '+ spades_output +'/contigs.fasta' +' > scaffold_output/minimap_out/mapping.paf' 

In [90]:
os.system(cmd_minimap)
df = pd.read_csv("scaffold_output/minimap_out/mapping.paf", sep ="\t", header=None)

[M::mm_idx_gen::0.201*1.01] collected minimizers
[M::mm_idx_gen::0.259*1.45] sorted minimizers
[M::main::0.259*1.45] loaded/built the index for 6 target sequence(s)
[M::mm_mapopt_update::0.291*1.41] mid_occ = 16
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 6
[M::mm_idx_stat::0.311*1.38] distinct minimizers: 940802 (97.51% are singletons); average occurrences: 1.044; average spacing: 5.351; total length: 5256088
[M::worker_pipeline::0.859*2.34] mapped 421 sequences
[M::main] Version: 2.24-r1122
[M::main] CMD: /home/vanhoan310/miniconda3/envs/amromics/bin/minimap2 /data/hoan/amromics/data/ncbi/Ecolitest/GCF_002860105.1_ASM286010v1_genomic.fna /data/hoan/amromics/simulation/art_output/spades_output_Ecolitest_GCF_002860105_v01/contigs.fasta
[M::main] Real time: 0.873 sec; CPU: 2.027 sec; Peak RSS: 0.083 GB


In [91]:
final_df = df.loc[df[5] == 'NZ_CP019071.1']
final_df = final_df.sort_values(by=[7])

In [92]:
# df

In [93]:
target_id = 0
gene_position = pd.read_csv(panta_output + '/gene_position.tsv', delimiter='\t', header=None)
gene_position.columns =['SampleID', 'ContigName', 'GeneSequence']
target_contigs = [gene_position.iloc[i, 1] for i in range(len(gene_position.index)) if gene_position.iloc[i, 0]==target_id]

In [94]:
target_contigs_list = [final_df.iloc[i, 0] for i in range(len(final_df.index)) if final_df.iloc[i, 0] in target_contigs]
target_contigs_sorted = [final_df.iloc[i, 0]+'---'+str(final_df.iloc[i, 7])+'>'+str(final_df.iloc[i, 8]) for i in range(len(final_df.index)) if final_df.iloc[i, 0] in target_contigs]

In [115]:
for i in range(len(target_contigs_list)-1):
    # print(i)
    # print(gene_position[gene_position.ContigName==target_contigs_list[i]].iloc[0,2].split(';')[-2:], end = '->')
    # print(gene_position[gene_position.ContigName==target_contigs_list[i+1]].iloc[0,2].split(';'))
    if gene_position[gene_position.ContigName==target_contigs_list[i]].iloc[0,2].split(';')[-1] in gene_position[gene_position.ContigName==target_contigs_list[i+1]].iloc[0,2].split(';'):
        print("Yes")

In [95]:
target_contigs_sorted

['NODE_46_length_30478_cov_18.766164---7>30478',
 'NODE_101_length_1318_cov_59.905780---30430>31738',
 'NODE_42_length_35714_cov_18.958440---31691>67397',
 'NODE_101_length_1318_cov_59.905780---67354>68662',
 'NODE_32_length_49096_cov_18.914826---68615>117705',
 'NODE_98_length_1442_cov_19.488104---118966>120397',
 'NODE_95_length_1592_cov_58.378660---121316>122896',
 'NODE_39_length_40147_cov_18.664796---124077>164215',
 'NODE_97_length_1443_cov_56.968300---164167>165605',
 'NODE_30_length_53138_cov_18.590415---165558>218680',
 'NODE_41_length_39638_cov_19.015209---219812>259446',
 'NODE_106_length_1120_cov_19.409390---260952>262061',
 'NODE_95_length_1592_cov_58.378660---262940>264520',
 'NODE_10_length_147945_cov_18.945473---265479>413412',
 'NODE_38_length_41324_cov_18.707698---413882>455196',
 'NODE_61_length_9714_cov_19.849570---460378>470090',
 'NODE_115_length_865_cov_55.879012---470046>470901',
 'NODE_34_length_44734_cov_18.805255---471041>515773',
 'NODE_87_length_2272_cov_97

In [96]:
# pangraph.adj_list0

In [97]:
# cmd_dnadiff = '~/miniconda3/envs/amromics/bin/dnadiff '+ref_data +' '+ spades_output +'/contigs.fasta -p dna_diff/'+sim_name
# os.system(cmd_dnadiff)

In [98]:
# Don't use SPADES, shovil keep files.
# python readSimulator/readSimulator.py --outdir shredded_reads  --input ncbi-genomes-2023-01-07/GCF_000240185.1_ASM24018v2_genomic.fna --simulator art --simulator_path art_bin_MountRainier/art_illumina --iterations 100 --readlen 100 --depth 100  --opts '-ss HS20 -sam -p -m 400 -s 60'
# investigate why we failed? #1/ connectivity in the assembly graph:
# multiplicity of each node: now 1, change to more: use length and coverage
# assembly graph: change to 500

In [99]:
def get_node_coverage(node_id):
    return float(node_id.split('_')[5])
get_node_coverage('NODE_45_length_30843_cov_18.832500')
target_contigs_multi_1 = [item for item in target_contigs_list if get_node_coverage(item) <= 25.0]

In [100]:
def get_value(edge_df0, source_id, target_id):
    # if source_id in edge_df0['source'].values and target_id in edge_df0['target'].values:
    #     print(edge_df0[((edge_df0['source'] == source_id) & (edge_df0['target'] == target_id))])
    # else:
    #     print('No value')
    df_res = edge_df0[((edge_df0['source'] == source_id) & (edge_df0['target'] == target_id))]
    if len(df_res.index) == 1:
        print(df_res.iloc[0, 2])
    else:
        print("No value")
# pangraph.edge_df0

def next_node_multi_1(target_contigs_list, idx):
    ## find the next node of multiplicity 1 after the index: idx
    current_idx = idx + 1
    while(get_node_coverage(target_contigs_list[current_idx]) >= 25.0):
        current_idx += 1
    return target_contigs_list[current_idx]

In [101]:
# target_contigs_multi_1

In [102]:
assembly_graph = pangraph.assembly_graph
for i in range(len(target_contigs_list)-1):
    print('---------------------- i =', i, '---------------------------------------------')
    node1 = target_contigs_list[i]
    node2 = target_contigs_list[i+1]
    print(node1, node2)
    source_id = 'C-' + str(pangraph.tail_contig[node1])
    target_id = 'C-' + str(pangraph.head_contig[node2]) 
    source_node = node1 + pangraph.strand[node1]
    target_node = node2 + pangraph.strand[node2]
    get_value(pangraph.edge_df0, node1, node2)
    if nx.has_path(pangraph.H, source_id, target_id):
        p = nx.shortest_path(pangraph.H, source=source_id, target=target_id)
        weight_p = 0.0
        for node_p_idx in range(len(p)-1):
            weight_p += pangraph.H[p[node_p_idx]][p[node_p_idx+1]]['weight']
        if len(p) >= 2:
            weight_p = 0.05 + weight_p/float((len(p)-1)*(len(p)-1))
        print(weight_p)
    else:
        print("No path in Pangraph")
    if assembly_graph.has_node(source_node) and assembly_graph.has_node(target_node):
        print("Is there a path in the assembly graph? ", end ='')
        print(nx.has_path(assembly_graph, source_node, target_node), end = ',')   
        if nx.has_path(assembly_graph, source_node, target_node):
            path_nucleotides = -1
            path = nx.shortest_path(assembly_graph, source=source_node, target=target_node)
            for idxp in range(1, len(path)-1):
                path_nucleotides = path_nucleotides + int(path[idxp].split("_")[3])
            print(len(path), path_nucleotides)
        else:
            print('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')
    if get_node_coverage(node2) >= 25.0:
        print('///////////////////////////////////////////////////////')
        node2 = next_node_multi_1(target_contigs_list, i+1)
        print(node1, node2)
        source_id = 'C-' + str(pangraph.tail_contig[node1])
        target_id = 'C-' + str(pangraph.head_contig[node2]) 
        source_node = node1 + pangraph.strand[node1]
        target_node = node2 + pangraph.strand[node2]
        get_value(pangraph.edge_df0, node1, node2)
        if nx.has_path(pangraph.H, source_id, target_id):
            p = nx.shortest_path(pangraph.H, source=source_id, target=target_id)
            weight_p = 0.0
            for node_p_idx in range(len(p)-1):
                weight_p += pangraph.H[p[node_p_idx]][p[node_p_idx+1]]['weight']
            if len(p) >= 2:
                weight_p = 0.05 + weight_p/float((len(p)-1)*(len(p)-1))
            print(weight_p)
        else:
            print("No path in Pangraph")
        if assembly_graph.has_node(source_node) and assembly_graph.has_node(target_node):
            print("Is there a path in the assembly graph? ", end ='')
            print(nx.has_path(assembly_graph, source_node, target_node), end = ',')   
            if nx.has_path(assembly_graph, source_node, target_node):
                path_nucleotides = -1
                path = nx.shortest_path(assembly_graph, source=source_node, target=target_node)
                for idxp in range(1, len(path)-1):
                    path_nucleotides = path_nucleotides + int(path[idxp].split("_")[3])
                print(len(path), path_nucleotides)
            else:
                print('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')
            
    
    #     # sum_len = 0
    #     # if nx.has_path(pangraph.H, source_id, target_id) or nx.has_path(assembly_graph, source_node, target_node):
    #     #     sum_len += int(final_df.iloc[i,1])
    #     #     print(sum_len, end='')
    # else:
    #     print("Restart............................................")
    #     sum_len = 0
    # # print('')

---------------------- i = 0 ---------------------------------------------
NODE_46_length_30478_cov_18.766164 NODE_101_length_1318_cov_59.905780
1.2750000000000001
No path in Pangraph
Is there a path in the assembly graph? True,2 -1
///////////////////////////////////////////////////////
NODE_46_length_30478_cov_18.766164 NODE_42_length_35714_cov_18.958440
11.353489583333335
20.05
Is there a path in the assembly graph? True,3 1317
---------------------- i = 1 ---------------------------------------------
NODE_101_length_1318_cov_59.905780 NODE_42_length_35714_cov_18.958440
1.2750000000000001
No path in Pangraph
Is there a path in the assembly graph? True,2 -1
---------------------- i = 2 ---------------------------------------------
NODE_42_length_35714_cov_18.958440 NODE_101_length_1318_cov_59.905780
1.2750000000000001
No path in Pangraph
Is there a path in the assembly graph? True,2 -1
///////////////////////////////////////////////////////
NODE_42_length_35714_cov_18.958440 NODE_32_

In [103]:
# pangraph.edge_df0

In [104]:
get_value(pangraph.edge_df0, 'NODE_94_length_1685_cov_19.542945', 'NODE_90_length_2072_cov_18.323748')

No value


In [105]:
# pangraph.edge_df0['source'].values